# Hyperparameter Tuning using HyperDrive

In the cell below, we import all the dependencies that we need to complete the project.

In [16]:
import azureml.core
from azureml.core import (
    Workspace,
    Experiment, 
    Dataset, 
    ComputeTarget,
    ScriptRunConfig,
    Environment,
)

from azureml.train.hyperdrive import (
    BanditPolicy,
    RandomParameterSampling,
    choice,
    uniform,
    loguniform,
    HyperDriveConfig,
    PrimaryMetricGoal,
)

from azureml.widgets import RunDetails
from azureml.core.model import InferenceConfig, Model
from azureml.core.webservice import AciWebservice
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal

import joblib
import uuid
import pandas as pd 
import requests
import json
import os

## Setup workspace and experiment

In [5]:
ws = Workspace.from_config()
experiment_name = 'capstone-experiment'
experiment=Experiment(ws, experiment_name)

print(f'Workspace name: {ws.name} / AZ region: {ws.location} ' \
    f'/ Subscription ID: {ws.subscription_id} / Resource group: {ws.resource_group}')

run = experiment.start_logging()

Workspace name: quick-starts-ws-239878 / AZ region: westeurope / Subscription ID: 9b72f9e6-56c5-4c16-991b-19c652994860 / Resource group: aml-quickstarts-239878


## Compute target assignment

In [11]:
from azureml.core.compute import AmlCompute

cluster_name = "capstone-cluster"

try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print(f"Found existing compute target: {compute_target}")
except Exception as e:
    print(f"Creating a new compute target (error: {e}")
    compute_cnfg = AmlCompute.provisioning_configuration(
        vm_size = "Standard_DS3_V2",
        min_nodes = 0,
        max_nodes = 4,
    )
    compute_target = ComputeTarget.create(
        ws,
        cluster_name,
        compute_cnfg,
    )
    compute_target.wait_for_completion(
        show_output=True,
        min_node_count=None,
        timeout_in_minutes=60,
    )

# message if ready
print(f'compute target: {compute_target.get_status().serialize()}')

Found existing compute target: AmlCompute(workspace=Workspace.create(name='quick-starts-ws-239878', subscription_id='9b72f9e6-56c5-4c16-991b-19c652994860', resource_group='aml-quickstarts-239878'), name=capstone-cluster, id=/subscriptions/9b72f9e6-56c5-4c16-991b-19c652994860/resourceGroups/aml-quickstarts-239878/providers/Microsoft.MachineLearningServices/workspaces/quick-starts-ws-239878/computes/capstone-cluster, type=AmlCompute, provisioning_state=Succeeded, location=westeurope, tags={})
compute target: {'currentNodeCount': 0, 'targetNodeCount': 0, 'nodeStateCounts': {'preparingNodeCount': 0, 'runningNodeCount': 0, 'idleNodeCount': 0, 'unusableNodeCount': 0, 'leavingNodeCount': 0, 'preemptedNodeCount': 0}, 'allocationState': 'Steady', 'allocationStateTransitionTime': '2023-08-07T14:48:59.724000+00:00', 'errors': None, 'creationTime': '2023-08-07T14:48:55.116214+00:00', 'modifiedTime': '2023-08-07T14:49:01.876533+00:00', 'provisioningState': 'Succeeded', 'provisioningStateTransitionT

## Dataset

For our capstone project, we use [the kaggle heart failure dataset](https://www.kaggle.com/datasets/andrewmvd/heart-failure-clinical-data). We uploaded and registered this dataset to the workspace beforehand. The dataset stems from [a publication on heart failure prediction using machine learning](https://bmcmedinformdecismak.biomedcentral.com/articles/10.1186/s12911-020-1023-5).

In [12]:
ds_name = 'heart_failure_kaggle'
dataset = Dataset.get_by_name(workspace=ws, name=ds_name)

In [13]:
# inspect the dataframe 
hf_prediction = dataset.to_pandas_dataframe()
hf_prediction.head()

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
0,75.0,0,582,0,20,1,265000.00,1.9,130,1,0,4,1
1,55.0,0,7861,0,38,0,263358.03,1.1,136,1,0,6,1
2,65.0,0,146,0,20,0,162000.00,1.3,129,1,1,7,1
3,50.0,1,111,0,20,0,210000.00,1.9,137,1,0,7,1
4,65.0,1,160,1,20,0,327000.00,2.7,116,0,0,8,1


In [14]:
hf_prediction.tail()

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
294,62.0,0,61,1,38,1,155000.0,1.1,143,1,1,270,0
295,55.0,0,1820,0,38,0,270000.0,1.2,139,0,0,271,0
296,45.0,0,2060,1,60,0,742000.0,0.8,138,0,0,278,0
297,45.0,0,2413,0,38,0,140000.0,1.4,140,1,1,280,0
298,50.0,0,196,0,45,0,395000.0,1.6,136,1,1,285,0


## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [20]:
# preliminaries
primary_metric_name = "accuracy"
if "training" not in os.listdir():
    os.mkdir("training/")

# setup training environment
task_env = Environment.from_pip_requirements(
    name="venv", 
    file_path="requirements.txt"
)

# policy specification
early_termination_policy = BanditPolicy(
    evaluation_interval=2, 
    slack_factor=0.2
)

# parameter sampler specification
param_sampling = RandomParameterSampling(
    {
        "--learning_rate": uniform(0.001, 0.5),
        "--n_estimators": choice(100, 150, 200, 300, 350),
    }
)

# estimator and hyperdrive config specification
estimator = ScriptRunConfig(
    source_directory=".",
    script="train.py",
    environment=task_env,
    compute_target=compute_target,
)

hyperdrive_run_config = HyperDriveConfig(
    run_config=estimator,
    hyperparameter_sampling=param_sampling,
    policy=early_termination_policy,
    primary_metric_name="Accuracy",
    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
    max_total_runs=10,
    max_concurrent_runs=4,
)

In [21]:
hyperdrive_run = experiment.submit(hyperdrive_run_config)

## Run Details

In the cell below, we use the `RunDetails` widget to show the results of the hyperdrive experiment defined above and look at the best model as well as its properties.

In [22]:
RunDetails(hyperdrive_run).show()
hyperdrive_run.wait_for_completion(show_output=True)

Bad pipe message: %s [b'\x16\xbb\x1f_\x19\x9a\xffo\xe1\xa2\xc0\x0f\xb9\x97\n:hZ \x93\x0c2#\x94%\x99\x9a\xf5\x94\xd0\xa5\xa8\xa8S\xba\x1c`\x0cXt\xf5s\x8d\xac/i\xf2\xab\xc3\xffp\x00\x08\x13\x02\x13\x03\x13\x01\x00\xff\x01\x00\x00\x8f\x00\x00']
Bad pipe message: %s [b'{\x80\xadt\x85\xa7~\x84\xcd\xc87t\xae\x06\xe9lj\xcb Y0t\xdb\x83\x07\x80Rlz2\xe2\x99\xb6\x03\xd7\xa7\x12!\xe7\xde\x0b\r\xde4CS\xbf\xc3pk\xde\x00\x08\x13', b'\x03', b'\x00\xff\x01\x00\x00\x8f\x00\x00\x00\x0e\x00\x0c\x00\x00\t127', b'.0.1\x00\x0b\x00\x04\x03\x00\x01\x02\x00\n\x00\x0c\x00\n\x00\x1d\x00\x17\x00\x1e\x00\x19\x00\x18\x00#\x00\x00\x00\x16\x00\x00\x00\x17\x00\x00\x00\r\x00\x1e\x00']
Bad pipe message: %s [b'\x03\x05\x03\x06\x03\x08\x07\x08\x08\x08\t\x08\n\x08\x0b\x08\x04\x08\x05\x08\x06\x04\x01\x05\x01\x06\x01']
Bad pipe message: %s [b"\x0bSu\xe3\xe7m=\x16\xd3t=\xe6\x8c\xe3\xa3L\x14\xb5\x00\x00|\xc0,\xc00\x00\xa3\x00\x9f\xcc\xa9\xcc\xa8\xcc\xaa\xc0\xaf\xc0\xad\xc0\xa3\xc0\x9f\xc0]\xc0a\xc0W\xc0S\xc0+\xc0/\x00\xa2\x00\x

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_04337308-ca63-4f41-9961-fa21e3761f62
Web View: https://ml.azure.com/runs/HD_04337308-ca63-4f41-9961-fa21e3761f62?wsid=/subscriptions/9b72f9e6-56c5-4c16-991b-19c652994860/resourcegroups/aml-quickstarts-239878/workspaces/quick-starts-ws-239878&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254

Streaming azureml-logs/hyperdrive.txt

[2023-08-07T21:48:25.558567][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space
[2023-08-07T21:48:26.1044384Z][SCHEDULER][INFO]Scheduling job, id='HD_04337308-ca63-4f41-9961-fa21e3761f62_0' 
[2023-08-07T21:48:26.3841873Z][SCHEDULER][INFO]Scheduling job, id='HD_04337308-ca63-4f41-9961-fa21e3761f62_2' 
[2023-08-07T21:48:26.2898822Z][SCHEDULER][INFO]Scheduling job, id='HD_04337308-ca63-4f41-9961-fa21e3761f62_1' 
[2023-08-07T21:48:26.4617720Z][SCHEDULER][INFO]Scheduling job, id='HD_04337308-ca63-4f41-9961-fa21e3761f62_3' 
[2023-08-07T21:48:26.382436][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution t

ActivityFailedException: ActivityFailedException:
	Message: Activity Failed:
{
    "error": {
        "code": "UserError",
        "message": "Execution failed. User process '/azureml-envs/azureml_58f440cd5db21b9ab3c363f087e7b355/bin/python' exited with status code 1. Please check log file 'user_logs/std_log.txt' for error details. Error: Traceback (most recent call last):\n  File \"<string>\", line 197, in <module>\n  File \"<string>\", line 193, in main\n  File \"/azureml-envs/azureml_58f440cd5db21b9ab3c363f087e7b355/lib/python3.8/runpy.py\", line 264, in run_path\n    code, fname = _get_code_from_file(run_name, path_name)\n  File \"/azureml-envs/azureml_58f440cd5db21b9ab3c363f087e7b355/lib/python3.8/runpy.py\", line 239, in _get_code_from_file\n    code = compile(f.read(), fname, 'exec')\n  File \"train.py\", line 55\n    joblib.dump(value=model, './outputs/model.joblib')\n                             ^\nSyntaxError: positional argument follows keyword argument\n\n Marking the exper

## Best Model

In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [ ]:
brun = hyperdrive_run.get_best_run_by_primary_metric()
print(f"ID for best model run: {brun.id}")

In [ ]:
brun_metrics = brun.get_metrics()
print(f"Best metrics collected from the best run: {brun_metrics}")
print(f"Reached accuracy: {brun_metrics['accuracy']}")

In [ ]:
# display best run details
print(brun)
brun

Then, we save the model and register it.

Additionally, we download the files from the run.

In [ ]:
brun.download_files()

## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [ ]:
model = brun.register_model(
    model_name="hyperdrive_model",
    model_path="./outputs/model.joblib"
)

TODO: In the cell below, send a request to the web service you deployed to test it.

In [ ]:
deployment_config = AciWebservice.deploy_configuration(
    cpu_cores=1,
    memory_gb=1,
)

inference_config = InferenceConfig(
    entry_script="score.py",
    environment=task_env,
)

service = Model.deploy(
    ws,
    "capstone-service",
    [model],
    inference_config,
    deployment_config,
    overwrite=True,
)

service.wait_for_deployment(show_output=True)

In the cell below, we test the service with some random rows from the dataset:

In [ ]:
target_var = "DEATH_EVENT"
json_payload = {
    "data": hf_prediction.drop(columns=target_var).sample(n=3).to_dict("records")
}

raw_data = json.dumps(json_payload)
print(raw_data)

In the cell below, we print the logs of the web service and delete the service

In [ ]:
print(service.get_logs())

Delete and shutdown webservice:

In [ ]:
service.delete()
model.delete()
compute_target.delete()

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.

